In [3]:
import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":"baltimore, md","page":"1","status_type":"ForRent"}

headers = {
    "X-RapidAPI-Key": "0f64ef61f3msh8589bcbe6519721p1fd4adjsn7b6792b52973",
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

json.load()

{'props': [{'units': [{'beds': '1', 'price': '$1,199+'}], 'listingStatus': 'FOR_RENT', 'zpid': '39.308994--76.62804', 'longitude': -76.62804, 'buildingName': 'Bolton St 1713', 'address': '1713 Bolton St, Baltimore, MD', 'detailUrl': '/b/bolton-st-1713-baltimore-md-5j8728/', 'isBuilding': True, 'hasImage': True, 'lotId': 1049911687, 'imgSrc': 'https://photos.zillowstatic.com/fp/e3819b051b082ceecf67c8c86e47360f-p_e.jpg', 'latitude': 39.308994}, {'units': [{'beds': '0', 'price': '$930+'}, {'beds': '1', 'price': '$1,347+'}], 'listingStatus': 'FOR_RENT', 'zpid': '39.289722--76.61699', 'longitude': -76.61699, 'buildingName': 'Vivo Living Baltimore', 'address': '101 W Fayette St, Baltimore, MD', 'detailUrl': '/apartments/baltimore-md/vivo-living-baltimore/9PcnGx/', 'isBuilding': True, 'hasImage': True, 'lotId': 2090479304, 'imgSrc': 'https://photos.zillowstatic.com/fp/a7568313a2fe28ec2a0df7d516227dac-p_e.jpg', 'latitude': 39.289722}, {'units': [{'beds': '0', 'price': '$830+'}, {'beds': '1', '

In [2]:
import requests
import time

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

headers = {
    "X-RapidAPI-Key": "0f64ef61f3msh8589bcbe6519721p1fd4adjsn7b6792b52973",
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

def extendedSearchQuery(current_page):
    querystring = {"location": "baltimore, md", "page": str(current_page), "status_type": "ForRent"}
    try:
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            return response
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

combined_data = {}
total_number_of_pages = 20
for i in range(total_number_of_pages):
    response = extendedSearchQuery(i + 1)
    if response is not None:
        page_data = response.json()
        
        # Check if 'props' exists in the response
        if "props" in page_data:
            page_props = page_data["props"]
            
            # Iterate through the props list on each page
            for prop in page_props:
                if "units" in prop:
                    # Get the units list
                    units = prop["units"]
                    
                    # Create temporary unit price keys based on the number of units
                    for idx, unit in enumerate(units):
                        num_bedrooms = int(unit['beds'])
                        price = unit['price']
                        key = f'unit_{num_bedrooms}_bedroom'
                        
                        # Update the prop dictionary with the individual unit keys
                        prop[key] = price
                    
                    # Remove the original "units" key
                    del prop["units"]
        
        combined_data["Page " + str(i + 1)] = page_data
    else:
        # Handle the case where the request failed
        combined_data["Page " + str(i + 1)] = "Request failed"

    if i % 2 == 0 and i <= total_number_of_pages:
        # If we have made 2 requests and it's not the last iteration, sleep for 1 second
        time.sleep(2)

Save file to json so that code doesn't need to make call to api everytime notebook is used

In [32]:
combined_data

file_path = "D:/DataSets/combined_data.json"  # Replace with your desired file path

with open(file_path, "w") as json_file:
    json.dump(combined_data, json_file)

In [7]:
file_path = "D:/DataSets/combined_data.json" 

with open(file_path, "r") as json_file:
    combined_data = json.load(json_file)
    


dict

In [33]:
combined_data

{'Page 1': {'props': [{'listingStatus': 'FOR_RENT',
    'zpid': '39.28158--76.61414',
    'longitude': -76.61414,
    'buildingName': 'Arrive Federal Hill',
    'address': '611 S Charles St, Baltimore, MD',
    'detailUrl': '/apartments/baltimore-md/arrive-federal-hill/9PdKvk/',
    'isBuilding': True,
    'hasImage': True,
    'lotId': 2090543288,
    'imgSrc': 'https://photos.zillowstatic.com/fp/56d7660181c7479269bef36e6f5c048c-p_e.jpg',
    'latitude': 39.28158,
    'unit_0_bedroom': '$1,765+',
    'unit_1_bedroom': '$1,685+',
    'unit_2_bedroom': '$2,424+'},
   {'listingStatus': 'FOR_RENT',
    'zpid': '39.288685--76.620094',
    'longitude': -76.620094,
    'buildingName': 'Redwood Campus Center',
    'address': '300 W Redwood St, Baltimore, MD',
    'detailUrl': '/apartments/baltimore-md/redwood-campus-center/ByN4cS/',
    'isBuilding': True,
    'hasImage': True,
    'lotId': 2532871615,
    'imgSrc': 'https://photos.zillowstatic.com/fp/4944c97ead4f910291e2064476797f10-p_e.jpg'

In [8]:
combined_data.keys()

dict_keys(['Page 1', 'Page 2', 'Page 3', 'Page 4', 'Page 5', 'Page 6', 'Page 7', 'Page 8', 'Page 9', 'Page 10', 'Page 11', 'Page 12', 'Page 13', 'Page 14', 'Page 15', 'Page 16', 'Page 17', 'Page 18', 'Page 19', 'Page 20'])

In [34]:
column_names = []
for i in range(total_number_of_pages):
    for prop in combined_data["Page " + str(i + 1)]["props"]:
        for key in prop.keys():
            if key not in column_names:  # Check if the key is not already in the list
                column_names.append(key)

# Rearrange column ordering
last_element = column_names.pop(10)
column_names.insert(3,last_element)

last_element = column_names.pop()
column_names.insert(15,last_element)

#Insert to GeoLocation feature
column_names.insert(4,"GeoLocation")

column_names

['listingStatus',
 'zpid',
 'longitude',
 'latitude',
 'GeoLocation',
 'buildingName',
 'address',
 'detailUrl',
 'isBuilding',
 'hasImage',
 'lotId',
 'imgSrc',
 'unit_0_bedroom',
 'unit_1_bedroom',
 'unit_2_bedroom',
 'unit_3_bedroom',
 'unit_4_bedroom',
 'dateSold',
 'propertyType',
 'lotAreaValue',
 'variableData',
 'zestimate',
 'price',
 'bedrooms',
 'contingentListingType',
 'listingSubType',
 'rentZestimate',
 'daysOnZillow',
 'bathrooms',
 'livingArea',
 'country',
 'currency',
 'lotAreaUnit',
 'unit']

In [62]:
import pandas as pd
df = pd.DataFrame(columns=column_names, dtype= str)

rows = []

for i in range(total_number_of_pages):
    for prop in combined_data["Page " + str(i + 1)]["props"]:
        rows_dict = {}
        for key, value in prop.items():
            rows_dict[key] = str(value)
        rows.append(rows_dict)
    
df = pd.DataFrame(rows)
pd.set_option('display.max_columns', None)

#Reposition unit_4_bedroom
last_column_name = df.columns[-1]
last_column_values = df.pop(last_column_name)
df.insert(15, last_column_name, last_column_values)

#Reposition longitude
last_column_name = df.columns[10]
last_column_values = df.pop(last_column_name)
df.insert(3, last_column_name, last_column_values)

df.insert(2,"GeoLocation", [f"{longitude}, {latitude}" for longitude, latitude in zip(df["longitude"], df["latitude"])])
df

,listingStatus,zpid,GeoLocation,longitude,latitude,buildingName,address,detailUrl,isBuilding,hasImage,lotId,imgSrc,unit_0_bedroom,unit_1_bedroom,unit_2_bedroom,unit_3_bedroom,unit_4_bedroom,dateSold,propertyType,lotAreaValue,variableData,zestimate,price,bedrooms,contingentListingType,listingSubType,rentZestimate,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,unit
0,FOR_RENT,39.28158--76.61414,"-76.61414, 39.28158",-76.61414,39.28158,Arrive Federal Hill,"611 S Charles St, Baltimore, MD",/apartments/baltimore-md/arrive-federal-hill/9...,True,True,2090543288,https://photos.zillowstatic.com/fp/56d7660181c...,"$1,765+","$1,685+","$2,424+",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOR_RENT,39.288685--76.620094,"-76.620094, 39.288685",-76.620094,39.288685,Redwood Campus Center,"300 W Redwood St, Baltimore, MD",/apartments/baltimore-md/redwood-campus-center...,True,True,2532871615,https://photos.zillowstatic.com/fp/4944c97ead4...,"$1,500+","$1,795+",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FOR_RENT,39.335617--76.6207,"-76.6207, 39.335617",-76.6207,39.335617,Broadview Apartments,"105 W 39th St, Baltimore, MD",/apartments/baltimore-md/broadview-apartments/...,True,True,1001416599,https://photos.zillowstatic.com/fp/078d5b0a63b...,"$1,270+","$1,540+","$2,750+",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FOR_RENT,39.292084--76.61599,"-76.61599, 39.292084",-76.61599,39.292084,The Apartments at Charles Plaza,"222 N Charles St, Baltimore, MD",/apartments/baltimore-md/the-apartments-at-cha...,True,True,1001416928,https://photos.zillowstatic.com/fp/759d2919183...,"$1,068+","$1,170+","$1,506+",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FOR_RENT,39.377323--76.66296,"-76.66296, 39.377323",-76.66296,39.377323,Copper Village and Quarry Village,"6617 Bonnie Ridge Dr, Baltimore, MD",/apartments/pikesville-md/copper-village-and-q...,True,True,1001416668,https://photos.zillowstatic.com/fp/4c51f468da9...,NaN,"$1,192+","$1,358+","$1,748+",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,FOR_RENT,2057188609,"-76.70322, 39.337494",-76.70322,39.337494,NaN,Haddon Hill Apartments (5512 -5514 Haddon Ave)...,/b/haddon-hill-apartments-%285512-5514-haddon-...,NaN,True,NaN,https://photos.zillowstatic.com/fp/41d3e934a90...,NaN,NaN,NaN,NaN,NaN,None,APARTMENT,None,None,None,995,2,None,{},1175,1,1,800,USA,USD,None,Apt T1
816,FOR_RENT,2057188611,"-76.70252, 39.337917",-76.70252,39.337917,NaN,Garden Style Apartment in Gwynn Oak/Seton Neig...,/b/garden-style-apartment-in-gwynn-oak-seton-n...,NaN,True,NaN,https://photos.zillowstatic.com/fp/800ce84f7be...,NaN,NaN,NaN,NaN,NaN,None,APARTMENT,None,None,None,975,2,None,{},1019,1,1,None,USA,USD,None,# 6
817,FOR_RENT,2057188612,"-76.679344, 39.326138",-76.679344,39.326138,NaN,Liberty Heights Apartments (3701 Liberty Heigh...,/b/liberty-heights-apartments-%283701-liberty-...,NaN,True,NaN,https://photos.zillowstatic.com/fp/b02dfaf922e...,NaN,NaN,NaN,NaN,NaN,None,APARTMENT,None,None,None,995,2,None,{},1132,1,1,800,USA,USD,None,# B2
818,FOR_RENT,39.3383--76.702324,"-76.702324, 39.3383",-76.702324,39.3383,Cadillac Apartments (5527 Cadillac Ave),"5527 Cadillac Ave, Gwynn Oak, MD",/b/cadillac-apartments-%285527-cadillac-ave%29...,True,True,1137512747,https://photos.zillowstatic.com/fp/20f9ed637ad...,NaN,NaN,$975+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [472]:
df.to_csv('D:/dataSets/property_data_extended.csv', index=False)

In [5]:
file = open("C:/Users/seanl/Datasets/sample2.json")

json.load(file)

{'firstName': 'Joe',
 'lastName': 'Jackson',
 'gender': 'male',
 'age': 28,
 'address': {'streetAddress': '101', 'city': 'San Diego', 'state': 'CA'},
 'phoneNumbers': [{'type': 'home', 'number': '7349282382'}]}

In [22]:
file = open("C:/Users/seanl/Datasets/sample2.json")

json.load(file)

file2 = open("C:/Users/seanl/Datasets/sample1.json")

data = json.load(file2)
# Use json.dumps with an indentation parameter to format the JSON data
formatted_json = json.dumps(data, indent=3)

print(formatted_json)

{
   "fruit": "Apple",
   "size": "Large",
   "color": "Red"
}


In [70]:
import json

# Open and load the first JSON file
file_path1 = "C:/Users/seanl/Datasets/sample2.json"
with open(file_path1, 'r') as file1:
    data1 = json.load(file1)

# Create an empty dictionary
combined_data = {}

# Add the data from the first JSON file twice
combined_data[0] = data1
combined_data[1] = data1

# Use json.dumps with an indentation parameter to format the combined JSON data
formatted_json = json.dumps(combined_data, indent=3)

print(formatted_json)

{
   "0": {
      "firstName": "Joe",
      "lastName": "Jackson",
      "gender": "male",
      "age": 28,
      "address": {
         "streetAddress": "101",
         "city": "San Diego",
         "state": "CA"
      },
      "phoneNumbers": [
         {
            "type": "home",
            "number": "7349282382"
         }
      ]
   },
   "1": {
      "firstName": "Joe",
      "lastName": "Jackson",
      "gender": "male",
      "age": 28,
      "address": {
         "streetAddress": "101",
         "city": "San Diego",
         "state": "CA"
      },
      "phoneNumbers": [
         {
            "type": "home",
            "number": "7349282382"
         }
      ]
   }
}
